In [1]:
import torch
import torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("cuda:", torch.cuda.is_available())


torch: 2.5.1+cu121
torchvision: 0.20.1+cu121
cuda: True


In [3]:
import os, copy, time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models


In [6]:
import os

print("CWD:", os.getcwd())
print("List ./ :", os.listdir("."))
print("List ./data :", os.listdir("data") if os.path.exists("data") else "NO data folder")


CWD: c:\Projects\Machine_Learning\notebooks\week3
List ./ : ['ants_and_bees_.ipynb', 'cifar_net.pth', 'classifer.ipynb', 'data']
List ./data : ['ants and bees', 'cifar-10-batches-py', 'cifar-10-python.tar.gz']


In [7]:
import os

for root, dirs, files in os.walk("."):
    if "hymenoptera_data" in dirs:
        print("FOUND:", os.path.join(root, "hymenoptera_data"))
        break

FOUND: .\data\ants and bees\hymenoptera_data


In [9]:
import os

data_dir = "data/ants and bees/hymenoptera_data"

print("exists:", os.path.exists(data_dir))
print("inside:", os.listdir(data_dir))
print("train:", os.listdir(os.path.join(data_dir, "train")))
print("val:", os.listdir(os.path.join(data_dir, "val")))


exists: True
inside: ['train', 'val']
train: ['ants', 'bees']
val: ['ants', 'bees']


In [11]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=(x=='train'), num_workers=2)
    for x in ['train', 'val']
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

print("classes:", class_names)
print("sizes:", dataset_sizes)


classes: ['ants', 'bees']
sizes: {'train': 244, 'val': 153}


In [12]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

for p in model.parameters():
    p.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)


device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\pirat/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


In [13]:
def train_model(model, epochs=5):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        for phase in ["train", "val"]:
            model.train() if phase=="train" else model.eval()

            running_loss, running_correct = 0.0, 0

            for x, y in dataloaders[phase]:
                x, y = x.to(device), y.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase=="train"):
                    out = model(x)
                    loss = criterion(out, y)
                    preds = out.argmax(1)

                    if phase=="train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * x.size(0)
                running_correct += (preds == y).sum().item()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_correct / dataset_sizes[phase]
            print(f"{phase}: loss={epoch_loss:.4f} acc={epoch_acc:.4f}")

train_model(model, epochs=5)



Epoch 1/5
train: loss=0.5820 acc=0.7131
val: loss=0.4476 acc=0.8431

Epoch 2/5
train: loss=0.4405 acc=0.8361
val: loss=0.3237 acc=0.9085

Epoch 3/5
train: loss=0.3544 acc=0.8893
val: loss=0.2853 acc=0.9150

Epoch 4/5
train: loss=0.2985 acc=0.9139
val: loss=0.2442 acc=0.9150

Epoch 5/5
train: loss=0.2882 acc=0.8852
val: loss=0.2202 acc=0.9281


In [14]:
torch.save(model.state_dict(), "resnet18_ants_bees.pth")
print("Saved resnet18_ants_bees.pth")


Saved resnet18_ants_bees.pth
